In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from langchain_ollama import OllamaLLM
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent

# 1. 시각화 설정
%matplotlib inline

# 한글 폰트 설정
plt.rcParams['font.family'] = 'NanumGothic'
plt.rcParams['axes.unicode_minus'] = False
sns.set_theme(style="whitegrid", font='NanumGothic') # Seaborn 테마 설정

# 2. 분석할 CSV 파일 로드 
file_path = "./input/dummy.csv" 
df = pd.read_csv(file_path)

print("데이터 로드 완료")
display(df.head()) 

# 3. 모델 및 에이전트 설정
model_name = "mistral-small"   # "mistral"  # "mistral-small"
llm = OllamaLLM(model=model_name, 
                temperature=0.0,
               )
llm_with_stop = llm.bind(stop=["Observation:", "Observation", "\nObservation"])  # 중지 시퀀스 설정
# 에이전트가 지켜야 할 철저한 규칙 정의
custom_prefix = """
You are a data analysis agent. 
You must use the following format:

Thought: your reasoning
Action: python_repl_ast
Action Input: your python code (WITHOUT ANY MARKDOWN BACKTICKS OR ```python TAGS)
Observation: the result
... (repeat)
Final Answer: final report in Korean

CRITICAL: Never use ```python or ``` blocks in Action Input. Just write raw code.
분석 결과(Observation)를 통해 필요한 통계 수치를 얻었다면, 
더 이상 코드를 실행하지 말고 즉시 'Final Answer:'를 사용하여 한국어 보고서를 작성하세요. 
반복적인 코드 실행보다 최종 답변 작성이 우선입니다.
"""

agent = create_pandas_dataframe_agent(
    llm_with_stop,  # llm,
    df,
    verbose=True,               # Thought/Action/Observation 실시간 출력
    allow_dangerous_code=True,  # 코드 실행 허용
    # 실행 엔진(AgentExecutor)에 전달할 인자를 별도 딕셔너리로 분리
    agent_executor_kwargs={
        "handle_parsing_errors": "Your output format is wrong. ALWAYS use 'Action: python_repl_ast' and 'Action Input: [code]'. DO NOT use markdown code blocks.",
        # "handle_parsing_errors": "Please provide your output in the correct format. Action must be 'python_repl_ast'. Action Input must be RAW python code without any markdown code blocks (no backticks).",
    },
    max_iterations=20, # 파싱 에러 시 재시도 횟수
    prefix=custom_prefix,
)

# 4. 분석 상세 요구 사항 입력 TODO 테스트용
summary_req = "전체적인 수치 요약해줘"
hypo_req = "플랫폼(CITY/GEN/GEN_OAS)별 차이나 요일별 특징에 대한 가설을 세워줘"
result_req = "가설을 검증하기 위해 실제 코드를 실행한 통계적 근거를 보여줘"
biz_req = "분석 결과를 바탕으로 한 분석 전략을 제안해줘"

# 5. 쿼리 조합
full_query = f"""
아래 요청사항에 맞춰 데이터프레임을 분석하고 한국어로 보고서를 작성하세요:

1. 데이터 요약: {summary_req}
2. 가설 설정: {hypo_req}
3. 분석 결과: {result_req}
4. 비즈니스 제언: {biz_req}

반드시 파이썬 코드를 실행하여 얻은 통계 수치를 바탕으로 답변하세요.

주의사항:
- 첫 번째 줄은 반드시 'Thought:'로 시작하세요.
- 설명을 하거나 인사말을 하지 마세요.
- 불필요한 서술이나 코드에 대한 설명(예: '이 함수는 ~입니다', '수정한 후 결과는 ~')을 절대 하지 마세요.
- 파이썬 코드를 실행해야 할 때는 반드시 'Action: python_repl_ast' 형식을 사용하고, 코드 블록(```python)을 직접 답변에 포함하지 마세요.
- 당신의 답변은 반드시 'Thought:', 'Action:', 'Action Input:', 'Observation:'의 형식을 엄격히 따라야 합니다.
- 평균(mean)이나 합계(sum)를 구할 때는 반드시 숫자형 컬럼만 선택하거나 `numeric_only=True` 옵션을 사용하세요.
  예: df.groupby('col').mean(numeric_only=True)
- 데이터에 문자열이나 날짜가 섞여 있을 수 있음을 항상 유의하세요.
- 시각화는 matplotlib이나 seaborn을 사용하세요.
- 한글 폰트(NanumGothic)가 이미 설정되어 있으니 그대로 사용하세요.
- 주피터 노트북 환경이므로 별도의 파일 저장 없이 plt.show()를 호출하여 그래프를 출력하세요.
- 코드 수정 제안이나 인사말 등 불필요한 텍스트를 출력하지 마세요. 바로 분석에 필요한 코드를 실행하세요.
- 분석 결과(Observation)를 얻기 전까지는 'Final Answer:'를 내지 마세요.
- 최종 답변을 낼 때만 'Final Answer:'를 사용하세요.
- 모든 설명은 한국어로 작성하세요.

[FATAL RULE]
- No conversational filler.
- Action Input must be RAW python code.
- Format:
Thought: (reasoning)
Action: python_repl_ast
Action Input: (code)

[실행 규칙]
- Action: 반드시 'python_repl_ast'라고 적으세요.
- Action Input: 마크다운 백틱(```)을 절대 사용하지 마세요. 
- 예시:
Action: python_repl_ast
Action Input: print(df.head())
"""

# 6. 실행
print(f"{model_name} AGENT GO!GO!GO!...\n")
try:
    response = agent.run(full_query)

    print("\n" + "="*50)
    print("최종 분석 보고서")
    print("="*50)
    print(response)
except Exception as e:
    print(f"오류 발생: {e}")

데이터 로드 완료


,datetime,TEMP,DAY_OF_WEEK,CITY,GEN,OAS,GEN_OAS,TOTAL,WEEK_FRI,WEEK_MON,WEEK_SAT,WEEK_SUN,WEEK_THU,WEEK_TUE,WEEK_WED,HOLIDAY,SUM_gsflow
0,2015.1.1 0:00,-4.34,THU,3245.206,1535.650,55.644,1591.294,4836.500,0,0,0,0,1,0,0,0,4836.500
1,2015.1.1 1:00,-5.11,THU,3050.383,1402.442,41.914,1444.356,4494.739,0,0,0,0,1,0,0,0,4494.739
2,2015.1.1 2:00,-5.53,THU,2832.828,1307.376,43.179,1350.555,4183.383,0,0,0,0,1,0,0,0,4183.383
3,2015.1.1 3:00,-5.90,THU,2804.450,1286.281,43.940,1330.221,4134.671,0,0,0,0,1,0,0,0,4134.671
4,2015.1.1 4:00,-6.40,THU,2938.364,1295.799,44.316,1340.115,4278.479,0,0,0,0,1,0,0,0,4278.479


mistral-small AGENT GO!GO!GO!...



> Entering new AgentExecutor chain...
Thought: 데이터프레임의 요약 통계를 확인하기 위해 `describe()` 메서드를 사용하여 전체적인 수치 요약을 얻어야 합니다.
Action: python_repl_ast
Action Input: df.describe(numeric_only=True)TypeError: NDFrame.describe() got an unexpected keyword argument 'numeric_only'Action: python_repl_ast
Action Input: df.describe(include='number')               TEMP          CITY           GEN           OAS       GEN_OAS  \
count  87622.000000  87624.000000  87624.000000  87624.000000  87624.000000   
mean      14.293447   2104.017913   1780.130985    627.319562   2407.450547   
std        9.990969   1073.541596    736.142749    358.884858    883.048465   
min      -15.020000    331.226000    160.566000     20.121000    202.749000   
25%        5.980000   1313.094250   1213.703000    349.839500   1775.678250   
50%       15.120000   1739.174500   1737.160500    647.917000   2374.312500   
75%       22.860000   2814.113750   2293.986000    896.706000   2995.936000   
max 

In [ ]:
import pandas as pd
from langchain_community.llms import Ollama   # # from langchain_ollama import OllamaLLM  # # TODO 차후 최신 OllamaLLM 로 변경 필요
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
# from langchain.agents import AgentType  # # from langchain.agents.agent_types import AgentType

# 1. 로컬 LLM 설정
# [중요] Agent가 정확한 코드 생성위해 temperature=0 설정
llm = Ollama(model="mistral-small", temperature=0)
# llm = OllamaLLM(model="mistral-small", temperature=0)

# 2. 분석위한 dummy data TODO csv나 DB에서 실데이터 불러오는 것으로 수정
data = {
    'date': pd.date_range(start='2025-10-01', periods=14),
    'visitor_count': [1200, 1300, 1100, 900, 850, 1500, 1600, 1150, 1050, 950, 800, 1450, 1550, 1100],
    'order_count': [120, 135, 115, 80, 75, 180, 195, 110, 100, 85, 70, 175, 190, 105],
    'ad_spend': [500, 500, 500, 300, 300, 700, 700, 500, 500, 500, 300, 700, 700, 500],
    'platform': ['PC', 'Mobile', 'Mobile', 'PC', 'PC', 'Mobile', 'Mobile', 'PC', 'Mobile', 'PC', 'PC', 'Mobile', 'Mobile', 'PC']
}
df = pd.DataFrame(data)

# 3. Pandas DataFrame Agent 생성
agent = create_pandas_dataframe_agent(
    llm,
    df,
    verbose=True, # Agent가 생각 과정(Thought/Action) 화면 출력
    # agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    allow_dangerous_code=True, # # 로컬 환경에서 LLM이 python 실행 허용 설정
)

# 4. 분석 요청 프롬프트 (Few-shot 구조 결합)
query = """
제공된 데이터프레임을 기반으로 다음 구조에 맞춰 한국어 보고서를 작성하세요:

1. 데이터 요약: 전체적인 수치(방문자, 주문수 등) 요약
2. 가설 설정: 플랫폼(PC/Mobile)별 효율 차이나 요일별 특징에 대한 가설
3. 분석 결과: 가설을 검증하기 위해 실제 코드를 실행한 통계적 근거 (예: 플랫폼별 전환율 비교)
4. 비즈니스 제언: 분석 결과를 바탕으로 한 마케팅 전략

반드시 코드를 실행하여 정확한 평균값과 상관관계를 계산한 뒤 답변하세요.
"""

print("Agent GO!GO!GO!...")
print("-" * 50)

try:
    response = agent.run(query)
    print("\n" + "="*50)
    print("최종 분석 보고서")
    print("="*50)
    print(response)
except Exception as e:
    print(f"Agent 실행 중 오류 발생: {e}")